In [2]:
import pandas as pd
import numpy as np
from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from sklearn.preprocessing import MaxAbsScaler

In [3]:
mmm_data = pd.read_csv('media_mix_model_final_data.csv')

In [6]:
mmm_data.columns

Index(['date_report', 'sp500', 'year', 'week', 'non_restricted_members',
       'affiliates', 'apple_search', 'google_search', 'bing_search', 'tiktok',
       'spotify', 'pinterest', 'buzzfeed', 'influencer', 'ooh', 'facebook',
       'instagram', 'audience_network', 'elle_raise', 'money_cheat_sheet',
       'opinionated', 'weekly_events', 'tofu_emails', 'instagram_followers',
       'instagram_new_followers', 'instagram_likes', 'instagram_comments',
       'Instagram_posts', 'instagram_impressions', 'instagram_reach',
       'instagram_bio_clicks', 'facebook_followers', 'facebook_new_followers',
       'linkedin_followers', 'linkedin_new_followers', 'twitter_followers',
       'twitter_new_followers', 'tiktok_followers', 'tiktok_new_followers',
       'campaigns', 'promos', 'campaigns_promos', 'interview',
       'other_appearance', 'podcast', 'speaking', 'tv'],
      dtype='object')

Combined columns to reduce variables in model

In [ ]:
mmm_data["paid_facebook"] = mmm_data["paid_facebook"]